In [1]:
import pandas_datareader as web
import pandas as pd
from pandas.tseries.offsets import BDay
from datetime import datetime as dt
from datetime import date, timedelta
import matplotlib.pyplot as plt
import eikon as ek
ek.set_app_key('f01a5002bf4649578cf705c13f7d30d9bdcf2c9d')

In [2]:
today = dt.today().strftime('%Y-%m-%d')

In [18]:
current_rate_fred = web.DataReader('DFEDTARU', 'fred', start = (pd.datetime.today() - BDay(2)).strftime('%Y-%m-%d'), end = dt.today().strftime('%Y-%m-%d'))
current_rate = current_rate_fred['DFEDTARU'][0] - 0.125 ###^^^^^^ DOESNT WORK WITH WEEKENDS OR HOLIDAYS^^^^####

In [67]:
meeting_dates = pd.read_csv(r'C:\Users\Keavan\Jupyter\Economics\fed_funds\fomc_meetings\fomc_meetings.csv', header = 0)
meeting_dates['date'] = pd.to_datetime(meeting_dates['date'], format='%Y-%m-%d')

date = str(list(meeting_dates['date'][meeting_dates['date'] >= today])[0])
next_meeting_date = dt.strptime(date, format('%Y-%m-%d %H:%M:%S'))
next_meeting_month = dt.strptime(date, format('%Y-%m-%d %H:%M:%S')).month

In [58]:
instruments = pd.read_csv(r'C:\Users\Keavan\Jupyter\Economics\fed_funds\ref\ff_codes.csv', header = 0)

In [69]:
ff_code = instruments['ric'].loc[instruments['month'] == next_meeting_month].iloc[0]

In [70]:
ff_code

'FFH0'

In [62]:
current_price = pd.DataFrame(ek.get_data(ff_code, fields = 'CF_LAST'))[0][0]['CF_LAST'][0]

In [63]:
implied_rate = 100 - current_price
cut = -0.25
hike = 0.25

In [64]:
prob_hike = (implied_rate - current_rate) / hike
prob_cut = (implied_rate - current_rate) / cut

In [71]:
next_meeting_date

datetime.datetime(2020, 3, 18, 0, 0)

In [78]:
ir_action_pr = max((prob_hike, prob_cut))
if ir_action_pr == prob_hike:
    action = 'hiking'
else:
    action = 'cutting'

print('The FOMC has a {:.2f}% chance of a {} overnight lending rates at the next meeting on {}/{}/{}.'.format(ir_action_pr * 100,
        action, next_meeting_date.day, next_meeting_date.month, next_meeting_date.year))

The FOMC has a 20.00% chance of a cutting overnight lending rates at the next meeting on 18/3/2020.
